# Mount drive and append path to PYTONPATH


In [ ]:
import os
import sys

from google.colab import drive

drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/DeepLCMS/train_google_colab")

# Import and install libraries

In [ ]:
%%capture
!pip install lightning
!pip install timm
!pip install torchinfo
!pip install scikit-posthocs
!pip install optuna
!pip install torchcam

In [ ]:
import colab_functions
import colab_utils
import pandas as pd
import prepare_data
import timm
import train_NN
from lightning.pytorch import loggers, callbacks, tuner, trainer
from pathlib import Path
from tqdm import tqdm
from IPython.display import clear_output
import seaborn as sns
import matplotlib.pyplot as plt
import scikit_posthocs as sp

In [ ]:
# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Unzip data

In [ ]:
!unzip -q "/content/drive/MyDrive/DeepLCMS/*.zip"

# Check if GPU is used

In [ ]:
device = colab_functions.get_device()

# Testing experimental conditions


In [ ]:
img_paths = [
    path for path in Path("./").rglob("ST001618_Opium_study_LC_MS*") if path.is_dir()
]
img_paths

In [ ]:
PRETRAINED_MODEL = "mobilevitv2_200.cvnets_in22k_ft_in1k_384"

model = train_NN.PretrainedModel(
    pretrained_model_name=PRETRAINED_MODEL, learning_rate=0.001
)
datamodule = prepare_data.LCMSDataModule(
    model,
    data_dir=img_paths[0],
)
model.show_architecture()

In [ ]:
for img_path in tqdm(img_paths):
    for round in range(1, 4):
        try:
            print(f"Round {round}, working on: {img_path}")
            model = train_NN.PretrainedModel(
                pretrained_model_name=PRETRAINED_MODEL, learning_rate=0.001
            )
            datamodule = prepare_data.LCMSDataModule(
                model,
                data_dir=img_path,
            )

            logger = loggers.CSVLogger("logs", name=str(img_path))

            trainer_ = trainer.Trainer(
                max_epochs=50,
                log_every_n_steps=1,
                logger=logger,
                precision="16-mixed",
                callbacks=[
                    callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=10)
                ],
            )

            trainer_.fit(model=model, datamodule=datamodule)

            del model, datamodule, trainer_
            clear_output(wait=True)

        except RuntimeError as e:
            print(f"{img_path} could not run because {e}")

results_df = colab_functions.get_experiment_results()
results_df.to_csv("pretrained_model_results.csv", index=False)

In [ ]:
results_df = colab_functions.get_experiment_results()
results_df.to_csv("pretrained_model_results.csv", index=False)

# Load saved result from disk


In [ ]:
# Load saved result from disk
results_df = pd.read_csv("pretrained_model_results.csv")
colab_functions.plot_experiment_results(results_df, bbox_to_anchor=(1.25, 0.97))

In [ ]:
pivoted_result = (
    results_df.pivot(index=["epoch", "experiment"], columns="variable", values="value")
    .reset_index()
    .sort_values(by="val_f1", ascending=False)
    .groupby("experiment")
    .head(1)
)
pivoted_result

In [ ]:
with sns.plotting_context("talk", font_scale=0.8):
    grid = sns.FacetGrid(
        (
            pivoted_result.drop(columns="epoch")
            .melt(id_vars="experiment")
            .replace(
                {
                    "ST001618_Opium_study_LC_MS_500": "500 images",
                    "ST001618_Opium_study_LC_MS_1000": "1000 images",
                    "ST001618_Opium_study_LC_MS_500_augmented": "500 images augmented",
                    "ST001618_Opium_study_LC_MS_1000_augmented": "1000 images augmented",
                }
            )
            .assign(variable=lambda df: df.variable.str.replace("_", " ").str.title())
        ),
        col="variable",
        col_wrap=5,
    )
    grid.map_dataframe(
        sns.barplot,
        y="experiment",
        x="value",
        capsize=0.15,
    )

    grid.set_titles(
        row_template="{row_name}", col_template="{col_name}", fontweight="bold", size=16
    )
    grid.set_axis_labels("", "")

    # Add labels to each bar
    for ax in grid.axes.flatten():
        for container in ax.containers:
            ax.bar_label(
                container,
                labels=[f"{x:.3f}" for x in container.datavalues],
                fontsize=10,
                padding=17,
            )

    plt.tight_layout()

    grid.savefig("experiment_result_1.png")